In [1]:
!pip install -q --upgrade pip
!pip install -U jaxlib==0.4.4+cuda11.cudnn82 -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
!pip install -q --upgrade jax[cuda11_cudnn805] -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
!pip install -q git+https://www.github.com/google/neural-tangents
!pip3 install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cpu
!pip install pyg-lib torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.13.0+cpu.html

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 44.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 45.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.2/111.2 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.7/79.7 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 87.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.2 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.d

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/Masterarbeit/two_WL_NTK

Mounted at /content/drive
/content/drive/MyDrive/Colab Notebooks/Masterarbeit/two_WL_NTK


In [2]:
%cd /content/drive/MyDrive/Colab Notebooks/Masterarbeit/two_WL_NTK

[Errno 2] No such file or directory: '/content/drive/MyDrive/Colab Notebooks/Masterarbeit/two_WL_NTK'
/content


In [2]:
# https://jax.readthedocs.io/en/latest/gpu_memory_allocation.html
%env XLA_PYTHON_CLIENT_PREALLOCATE=false
#%env JAX_PLATFORMS=cpu
%env XLA_PYTHON_CLIENT_ALLOCATOR=platform

env: XLA_PYTHON_CLIENT_PREALLOCATE=false
env: XLA_PYTHON_CLIENT_ALLOCATOR=platform


In [3]:
from utils import *
import os

import neural_tangents as nt
from neural_tangents import stax
from jax import numpy as np
from jax import random
import jax
from jax import numpy as np

In [4]:
# load preprocessed data

dataset_name = "MUTAG"
base_path = f"/content/drive/MyDrive/MasterarbeitData/Preprocessed/{dataset_name}"

ys = np.load(base_path + f"/ys.npy")
graps_node_features = np.load(base_path + f"/graps_node_features.npy")
graph_conv_pattern = np.load(base_path + f"/graph_conv_pattern.npy")

# need to add a copy at axis 2 to make the argegation over axis (1,2) work
# need to aggreagte over axis (1,2) to keep the shape of the kernel stable in the aggregation layer
graph_conv_pattern = np.append(graph_conv_pattern, graph_conv_pattern, axis=2)

print(graps_node_features.shape) # (188, 28, 1, 7)
print(graph_conv_pattern.shape) # (188, 66, 1, 2)

(188, 28, 7, 1)
(188, 66, 2, 2)


In [5]:
layer_wide = 16
# define a grap convolution network
init_fn, apply_fn, kernel_fn = stax.serial(
    stax.Conv(layer_wide, (1,1), parameterization="ntk"), stax.Relu(),
    stax.Aggregate(aggregate_axis=(1,2), batch_axis=0, channel_axis=3, implementation="SPARSE"),
    stax.Conv(layer_wide, (1,1), parameterization="ntk"), stax.Relu(),
    stax.Aggregate(aggregate_axis=(1,2), batch_axis=0, channel_axis=3, implementation="SPARSE"),
    # stax.Conv(layer_wide, (1,1), parameterization="ntk"), stax.Relu(),
    # stax.Aggregate(aggregate_axis=(1,2), batch_axis=0, channel_axis=3, implementation="SPARSE"),
    stax.GlobalSumPool(),
    stax.Dense(1),
)

/usr/local/lib/python3.8/dist-packages/neural_tangents/_src/stax/linear.py:439: UserWarning: Negative indices in `pattern` are considered as padding (i.e. ignored), unlike typical numpy negative indexing.
  warnings.warn('Negative indices in `pattern` are considered as padding '


In [9]:
# calculate the kernel range wise 
size = 50
dataset_size = 188
r = 3

base_path = f"/content/drive/MyDrive/MasterarbeitData"
data_path = base_path + f"/Preprocessed/MUTAG"
split_data_path = data_path + f"/splits/size_{size}_range_{r}" 
kernel_path = base_path + f"/Kernels/MUTAG/GCNN"
if not os.path.exists(kernel_path):
    os.makedirs(kernel_path)

for i in range(r+1):
  for j in range(r+1):
    
    l = lambda x : x*size 
    h = lambda x : (1+x)*size if (not x == r) else dataset_size-1

    x1 = graps_node_features[l(i):h(i),:]
    x2 = graps_node_features[l(j):h(j),:]
    p1 = graph_conv_pattern[l(i):h(i),:]
    p2 = graph_conv_pattern[l(j):h(j),:]

    kernel_matrix = kernel_fn(x1, x2, 'ntk', pattern=(p1, p2))
    np.save(kernel_path + f"/NTK_{h(i)}_{h(j)}", kernel_matrix)

(37, 28, 7, 1)

In [13]:
kernel_matrix.shape

(37, 37)

In [10]:
kernel_matrix[-3:,-3:]

Array([[2116800., 2116800.,  478800.],
       [2116800., 2116800.,  478800.],
       [ 478800.,  478800.,  108300.]], dtype=float32)

In [12]:
tx1 = np.expand_dims(graps_node_features[-3,:], 0)
tx2 = np.expand_dims(graps_node_features[-3,:], 0)
tp1 = np.expand_dims(graph_conv_pattern[-3,:], 0)
tp2 = np.expand_dims(graph_conv_pattern[-3,:], 0)

kernel_fn(tx1, tx2, 'ntk', pattern=(tp1, tp2))

Array([[2116800.]], dtype=float32)

In [13]:
tx1 = np.expand_dims(graps_node_features[-4,:], 0)
tx2 = np.expand_dims(graps_node_features[-3,:], 0)
tp1 = np.expand_dims(graph_conv_pattern[-4,:], 0)
tp2 = np.expand_dims(graph_conv_pattern[-3,:], 0)

kernel_fn(tx1, tx2, 'ntk', pattern=(tp1, tp2))

Array([[2116800.]], dtype=float32)

In [17]:
np.squeeze(graps_node_features[-3,:]) == np.squeeze(graps_node_features[-4,:])

Array([[ True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True],
       [ True, False,  True, False,  True,  True,  True],
       [ True, False, False,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True],
       [False,  True, False,  True,  True,  True,  True],
       [False, False,  True,  True,  True,  True,  True],
       [ True, False, False,  True,  True,  True,  True],
       [ True,  True, False,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True],
       [ True,

In [15]:
np.squeeze(graps_node_features[-4,:])

Array([[1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.],
 

In [28]:
np.squeeze(x1[-3,:]) == np.squeeze(x1[-2,:])

Array([[ True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True],
       [ True, False,  True, False,  True,  True,  True],
       [ True, False, False,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True],
       [False,  True, False,  True,  True,  True,  True],
       [False, False,  True,  True,  True,  True,  True],
       [ True, False, False,  True,  True,  True,  True],
       [ True,  True, False,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True,  True],
       [ True,